### SparkSession

In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('sql') \
    .getOrCreate()

### Reading

In [ ]:
df_green = spark.read.parquet('data/parquet/green/*/*/')
df_yellow = spark.read.parquet('data/parquet/yellow/*/*')

### modify & combine columns

In [ ]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [ ]:
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [ ]:
# get common columns btw green & yellow DataFrame
common_columns = []

for col in df_green.columns:
    if col in df_yellow.columns:
        common_columns.append(col)

In [ ]:
from pyspark.sql import functions as F

In [ ]:
df_green_sel = df_green.select(common_columns).withColumn('service_type', F.lit('green'))
df_yellow_sel = df_yellow.select(common_columns).withColumn('service_type', F.lit('yellow'))

In [ ]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

### groupby

In [ ]:
df_trips_data.groupBy('service_type').count().show()

### SQL

In [ ]:
df_trips_data.createOrReplaceTempView('trips_data')

In [ ]:
spark.sql("""
    SELECT
        service_type,
        count(1)
    FROM
        trips_data
    GROUP BY
        service_type
""").show()

In [ ]:
df_result = spark.sql("""
SELECT
    -- Revenue grouping
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_monthly_passenger_count,
    AVG(trip_distance) AS avg_monthly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [ ]:
df_result.coalesce(1).write.parquet('data/report/revenue/', mode='overwrite')